# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 9:39AM,240823,10,0085814255,ISDIN Corporation,Released
1,Jun 30 2022 9:39AM,240823,10,0085814256,ISDIN Corporation,Released
2,Jun 30 2022 9:39AM,240823,10,0085814272,ISDIN Corporation,Released
3,Jun 30 2022 9:39AM,240823,10,0085814271,ISDIN Corporation,Released
4,Jun 30 2022 9:39AM,240823,10,0085814279,ISDIN Corporation,Released
5,Jun 30 2022 9:39AM,240823,10,0085814284,ISDIN Corporation,Released
6,Jun 30 2022 9:39AM,240823,10,0085814306,ISDIN Corporation,Released
7,Jun 30 2022 9:39AM,240823,10,0085814315,ISDIN Corporation,Released
8,Jun 30 2022 9:39AM,240823,10,0085814314,ISDIN Corporation,Released
9,Jun 30 2022 9:39AM,240823,10,0085814385,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,240819,Released,16
38,240820,Released,2
39,240821,Released,5
40,240823,Released,50
41,240826,Released,18


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240819,NaN,NaN,16.0
240820,NaN,NaN,2.0
240821,NaN,NaN,5.0
240823,NaN,NaN,50.0
240826,NaN,NaN,18.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240745,1.0,11.0,9.0
240750,0.0,25.0,1.0
240763,1.0,2.0,3.0
240766,0.0,0.0,1.0
240768,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240745,1,11,9
240750,0,25,1
240763,1,2,3
240766,0,0,1
240768,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240745,1,11,9
1,240750,0,25,1
2,240763,1,2,3
3,240766,0,0,1
4,240768,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240745,1,11,9
1,240750,,25,1
2,240763,1,2,3
3,240766,,,1
4,240768,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation
50,Jun 30 2022 9:37AM,240826,20,Alumier Labs Inc.
68,Jun 30 2022 9:30AM,240807,10,ISDIN Corporation
119,Jun 30 2022 9:27AM,240821,10,Emerginnova
124,Jun 30 2022 9:25AM,240819,10,Bio-PRF
140,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc."
142,Jun 30 2022 9:22AM,240818,16,Sartorius Stedim Filters Inc.
143,Jun 30 2022 9:20AM,240817,10,Ivaoes Animal Health
144,Jun 30 2022 9:19AM,240816,10,"Nextsource Biotechnology, LLC"
146,Jun 30 2022 9:09AM,240813,18,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation,,,50
1,Jun 30 2022 9:37AM,240826,20,Alumier Labs Inc.,,,18
2,Jun 30 2022 9:30AM,240807,10,ISDIN Corporation,,,51
3,Jun 30 2022 9:27AM,240821,10,Emerginnova,,,5
4,Jun 30 2022 9:25AM,240819,10,Bio-PRF,,,16
5,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc.",,,2
6,Jun 30 2022 9:22AM,240818,16,Sartorius Stedim Filters Inc.,,,1
7,Jun 30 2022 9:20AM,240817,10,Ivaoes Animal Health,,,1
8,Jun 30 2022 9:19AM,240816,10,"Nextsource Biotechnology, LLC",,1,1
9,Jun 30 2022 9:09AM,240813,18,"ClearSpec, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation,50,,
1,Jun 30 2022 9:37AM,240826,20,Alumier Labs Inc.,18,,
2,Jun 30 2022 9:30AM,240807,10,ISDIN Corporation,51,,
3,Jun 30 2022 9:27AM,240821,10,Emerginnova,5,,
4,Jun 30 2022 9:25AM,240819,10,Bio-PRF,16,,
5,Jun 30 2022 9:24AM,240820,20,"Exact-Rx, Inc.",2,,
6,Jun 30 2022 9:22AM,240818,16,Sartorius Stedim Filters Inc.,1,,
7,Jun 30 2022 9:20AM,240817,10,Ivaoes Animal Health,1,,
8,Jun 30 2022 9:19AM,240816,10,"Nextsource Biotechnology, LLC",1,1,
9,Jun 30 2022 9:09AM,240813,18,"ClearSpec, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation,50,,
1,Jun 30 2022 9:37AM,240826,20,Alumier Labs Inc.,18,,
2,Jun 30 2022 9:30AM,240807,10,ISDIN Corporation,51,,
3,Jun 30 2022 9:27AM,240821,10,Emerginnova,5,,
4,Jun 30 2022 9:25AM,240819,10,Bio-PRF,16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation,50.0,NaN,NaN
1,Jun 30 2022 9:37AM,240826,20,Alumier Labs Inc.,18.0,NaN,NaN
2,Jun 30 2022 9:30AM,240807,10,ISDIN Corporation,51.0,NaN,NaN
3,Jun 30 2022 9:27AM,240821,10,Emerginnova,5.0,NaN,NaN
4,Jun 30 2022 9:25AM,240819,10,Bio-PRF,16.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 9:39AM,240823,10,ISDIN Corporation,50.0,0.0,0.0
1,Jun 30 2022 9:37AM,240826,20,Alumier Labs Inc.,18.0,0.0,0.0
2,Jun 30 2022 9:30AM,240807,10,ISDIN Corporation,51.0,0.0,0.0
3,Jun 30 2022 9:27AM,240821,10,Emerginnova,5.0,0.0,0.0
4,Jun 30 2022 9:25AM,240819,10,Bio-PRF,16.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2408093,145.0,25.0,0.0
12,481619,2.0,1.0,0.0
16,722424,6.0,3.0,0.0
17,240768,1.0,0.0,0.0
18,240813,1.0,0.0,0.0
19,722373,46.0,15.0,0.0
20,1685430,35.0,63.0,2.0
21,1203910,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2408093,145.0,25.0,0.0
1,12,481619,2.0,1.0,0.0
2,16,722424,6.0,3.0,0.0
3,17,240768,1.0,0.0,0.0
4,18,240813,1.0,0.0,0.0
5,19,722373,46.0,15.0,0.0
6,20,1685430,35.0,63.0,2.0
7,21,1203910,4.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,145.0,25.0,0.0
1,12,2.0,1.0,0.0
2,16,6.0,3.0,0.0
3,17,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,46.0,15.0,0.0
6,20,35.0,63.0,2.0
7,21,4.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,145.0
1,12,Released,2.0
2,16,Released,6.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
Executing,25.0,1.0,3.0,0.0,0.0,15.0,63.0,1.0
Released,145.0,2.0,6.0,1.0,1.0,46.0,35.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,Executing,25.0,1.0,3.0,0.0,0.0,15.0,63.0,1.0
2,Released,145.0,2.0,6.0,1.0,1.0,46.0,35.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,Executing,25.0,1.0,3.0,0.0,0.0,15.0,63.0,1.0
2,Released,145.0,2.0,6.0,1.0,1.0,46.0,35.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()